In [1]:
from jenga.tasks.openml import OpenMLTask
from jenga.corruptions.generic import MissingValues
    
import pandas as pd
import numpy as np

from data_imputation_paper.imputation.simple import ModeImputer
from data_imputation_paper.imputation.ml import KNNImputer, ForestImputer
from data_imputation_paper.imputation.generative import GAINImputer
from data_imputation_paper.evaluation import Evaluator

## Make Deterministic

In [2]:
seed = 42

## Create example tasks

In [3]:
task = OpenMLTask(seed=seed, openml_id=4552)

if task.contains_missing_values():
    raise ValueError("This would distort the evaluation because we wouldn't have a full ground truth.")

Found 14 categorical columns: ['V1', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15']
Found 2 numeric columns: ['V2', 'V16']


## Insert missing values using jenga

In [4]:
missing_values = [
    MissingValues(column='V2', fraction=0.5, na_value=np.nan, missingness='MCAR'),
    MissingValues(column='V4', fraction=0.5, na_value=np.nan, missingness='MCAR'),
    MissingValues(column='V15', fraction=0.5, na_value=np.nan, missingness='MCAR')
]

## Mode Imputation

In [5]:
arguments = {
    "seed": seed
}

%time Evaluator(task, missing_values, ModeImputer, arguments).evaluate(10).report()

Evaluation result contains 3 target columns: V2, V4, V15
All are in a round-robin fashion imputed and performances are as follows:

Target Column: V2
            train         test
MAE     29.954967    29.193205
MSE   1411.781041  1396.991674
RMSE    37.573675    37.376352

Target Column: V4
                train      test
F1_micro     0.823036  0.844523
F1_macro     0.451465  0.457854
F1_weighted  0.743143  0.773337

Target Column: V15
                train      test
F1_micro     0.124007  0.088339
F1_macro     0.013791  0.010823
F1_weighted  0.027362  0.014341

CPU times: user 2.22 s, sys: 26.4 ms, total: 2.25 s
Wall time: 2.25 s


## KNN imputation

In [6]:
arguments = {
    "seed": seed,
    "hyperparameter_grid_categorical_imputer": {
        "n_neighbors": [3, 5]
    },
    "hyperparameter_grid_numerical_imputer": {
        "n_neighbors": [3, 5]
    }
}

%time Evaluator(task, missing_values, KNNImputer, arguments).evaluate(5).report()

Evaluation result contains 3 target columns: V2, V4, V15
All are in a round-robin fashion imputed and performances are as follows:

Target Column: V2
            train         test
MAE     30.697087    29.970671
MSE   1479.275375  1390.832155
RMSE    38.461349    37.293862

Target Column: V4
                train      test
F1_micro     0.932921  0.957597
F1_macro     0.880103  0.918508
F1_weighted  0.931517  0.957398

Target Column: V15
                train      test
F1_micro     0.501765  0.549470
F1_macro     0.397952  0.412726
F1_weighted  0.494373  0.540974

CPU times: user 11.3 s, sys: 2.76 s, total: 14.1 s
Wall time: 12.5 s


## Forest imputation

In [7]:
arguments = {
    "seed": seed,
    "hyperparameter_grid_categorical_imputer": {
        "n_estimators": [50, 100]
    },
    "hyperparameter_grid_numerical_imputer": {
        "n_estimators": [50, 100]
    }
}

%time Evaluator(task, missing_values, ForestImputer, arguments).evaluate(5).report()

Evaluation result contains 3 target columns: V2, V4, V15
All are in a round-robin fashion imputed and performances are as follows:

Target Column: V2
            train         test
MAE     28.146965    28.328991
MSE   1258.807218  1318.998384
RMSE    35.478678    36.316192

Target Column: V4
                train      test
F1_micro     0.955958  0.951237
F1_macro     0.919897  0.900870
F1_weighted  0.955054  0.950382

Target Column: V15
                train      test
F1_micro     0.550927  0.553004
F1_macro     0.463676  0.420468
F1_weighted  0.546331  0.552162

CPU times: user 36.4 s, sys: 1.27 s, total: 37.7 s
Wall time: 30.9 s


## GAIN imputation

In [8]:
arguments = {
    "num_data_columns": task.train_data.shape[1],
    "seed": seed,
    "hyperparameter_grid": {
        "gain": {
            "alpha": [80, 120],
            "hint_rate": [0.5, 0.9],
            "noise": [0.001, 0.1]
        },
        "training": {
            "batch_size": [64, 256],
            "epochs": [5, 15]
        }
    }
}

%time Evaluator(task, missing_values, GAINImputer, arguments).evaluate(2).report()

OSError: SavedModel file does not exist at: .model/{saved_model.pbtxt|saved_model.pb}